In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

In [2]:
import tensorflow as tf

2023-03-26 17:56:56.433676: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
from sklearn.preprocessing import LabelEncoder

In [4]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics
from datetime import datetime

# Dataset information - 

 ### Transaction Dataset information
- TransactionDT: timedelta from a given reference datetime (not an actual timestamp)
- TransactionAMT: transaction payment amount in USD
- ProductCD: product code, the product for each transaction
- card1 - card6: payment card information, such as card type, card category, issue bank, country, etc.
- addr: address
- dist: distance
- P_ and (R__) emaildomain: purchaser and recipient email domain
- C1-C14: counting, such as how many addresses are found to be associated with the payment card, etc. The actual meaning is masked.
- D1-D15: timedelta, such as days between previous transaction, etc.
- M1-M9: match, such as names on card and address, etc.
- Vxxx: Vesta engineered rich features, including ranking, counting, and other entity relations.
### Categorical Features:
- ProductCD
- card1 - card6
- addr1, addr2(billing region, country)
- P_emaildomain
- R_emaildomain
- M1 - M9

In [6]:
# This turns all the axes white in all the matplotlib plots. Comment this out if you dont want that
COLOR = 'white'
matplotlib.rcParams['text.color'] = COLOR
matplotlib.rcParams['axes.labelcolor'] = COLOR
matplotlib.rcParams['xtick.color'] = COLOR
matplotlib.rcParams['ytick.color'] = COLOR

In [7]:
df_transaction = pd.read_csv('/Users/abraryaser/Desktop/datasets/ieee-fraud-detection/train_transaction.csv')

df_transaction.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
# Features that are used, isFraud is the target
features = ['isFraud', 'TransactionDT', 'card4', 
            'TransactionAmt','ProductCD', 'P_emaildomain','R_emaildomain']


In [13]:
df = df_transaction[features]
df.head()

,isFraud,TransactionDT,card4,TransactionAmt,ProductCD,P_emaildomain,R_emaildomain
0,0,86400,discover,68.5,W,NaN,NaN
1,0,86401,mastercard,29.0,W,gmail.com,NaN
2,0,86469,visa,59.0,W,outlook.com,NaN
3,0,86499,mastercard,50.0,W,yahoo.com,NaN
4,0,86506,mastercard,50.0,H,gmail.com,NaN


In [14]:
target = 'isFraud'
# Categorical features
cat = ['TransactionDT','ProductCD', 'card4', 'P_emaildomain','R_emaildomain']
# Numeric features
num = ['TransactionAmt']

In [15]:
#Drop rows with missing features
df = df.dropna()
y = df[target].values

In [17]:
# Subset the dataframe rows or columns according to the specified index labels.
x_cat = df.filter(items = cat).values
x_num = df.filter(items = num).values

In [18]:
labelencoder_X = LabelEncoder()
# Label encode every categorical column
for i in range(len(cat)): 
    x_cat[:, i] = labelencoder_X.fit_transform(x_cat[:, i])

In [19]:
# Build input vector X, the training data
X = np.concatenate((x_cat, x_num), axis=1)

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2, random_state = 0)

In [25]:
X_train = np.asarray(X_train).astype('float32')
y_train = np.asarray(y_train).astype('float32').reshape((-1,1))
X_val = np.asarray(X_val).astype('float32')
y_val = np.asarray(y_val).astype('float32').reshape((-1,1))

In [26]:
X_train.shape, y_train.shape

((80656, 6), (80656, 1))

### Train Model
Model:
Input -> Dense(Linear + activation) -> Dense -> Dense -> Output

In [27]:
# Initialize model
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(1, activation=tf.nn.sigmoid))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=3)

2023-03-26 18:38:36.511445: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/3
2521/2521 [==============================] - 3s 1ms/step - loss: 80.1753 - accuracy: 0.8386
Epoch 2/3
2521/2521 [==============================] - 3s 1ms/step - loss: 17.9275 - accuracy: 0.8426
Epoch 3/3
2521/2521 [==============================] - 3s 1ms/step - loss: 5.4974 - accuracy: 0.8447


In [28]:
val_loss, val_acc = model.evaluate(X_val, y_val)
print(val_loss, val_acc)

631/631 [==============================] - 1s 727us/step - loss: 4.3215 - accuracy: 0.9204
4.321479320526123 0.9204027056694031
